In [1]:
# importing the required libraries
import pandas as pd
import numpy as np
import boto3
import seaborn as sns
import matplotlib.pyplot as plt
from src.utils.utils import *
from src.components.data_preprocessing import DataPreProcessing
from src.components.embedding_generation import EmbeddingGeneration
from src.components.annoy_indexing import AnnoyIndexer


import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='seaborn')
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prady\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# calling the function to downloading data
get_data_from_s3(path='../data/raw.csv',filename='raw.csv')

In [3]:
# 1. Preprocess the data
obj=DataPreProcessing()
preprocessed_data_path = obj.initiate_data_processing("../data/raw.csv")

In [4]:
# 2. Generate embeddings
embedding_generation = EmbeddingGeneration()
embeddings_path = embedding_generation.initiate_embedding_generation(processed_data_path)

In [5]:
# 3. Build Annoy Index
indexing = AnnoyIndexer()
index_path = indexing.build_index(embeddings_path)

In [12]:
# 4. Searching
from pprint import pprint
from src.components.annoy_indexing import AnnoyIndexer
from src.pipeline.search_pipeline import SearchPipeline

index = AnnoyIndexer()
index.load_index('../artifacts/annoy_index.ann')

# loading the metadata about articles
with open('../artifacts/metadata.json','r') as file:
    metadata = json.load(file)

# Load the data from the pickle file
with open('../artifacts/section_article_mapping.pkl', 'rb') as file:
    mapping = pickle.load(file)


# queries = [lookup[i]["title"] for i in (0, 1)]
query = ["women entrepreneuship in india"]
k=3


# searching for the relevant articles
searcher = SearchPipeline()
results = searcher.search(index, query, k, mapping, metadata)
pprint(results)


[{'query': 'women entrepreneuship in india',
  'results': [{'article_id': 860,
               'category': 'business, finance & economics',
               'score': 0.9434589743614197,
               'subcategory': 'business - female specific',
               'title': 'Meet Kunj Yadav, an entrepreneur inspiring women to '
                        'achieve their dreams'},
              {'article_id': 11030,
               'category': 'trade & professional',
               'score': 0.9397159814834595,
               'subcategory': 'crime, police & courts',
               'title': 'Indian police use facial recognition to persecute '
                        'Muslims and other marginalized communities'},
              {'article_id': 4340,
               'category': 'general news & current affairs',
               'score': 0.8683447241783142,
               'subcategory': 'feminism',
               'title': 'The women’s burden: Real stories bring alive the '
                        'struggles t